### Host linkages:
We have 38,956 viral sequences (vOTUs), but what is the point of all these if we don't link to hosts?
Would like to link the viruses to hosts, and get AMG predictions if possible

After host linkages: Would like to do co-occurence calculations for viruses and their hosts (sourmash?) to see if the linkages make sense. 

How to:
- Use Minced to predict repeat-spacer areas
- For MAGs with a predicted repeat-spacer area, run Prokka for the cas gene predictions.


Tried:
- iphop
- crisprcastyper
- crisprcasfinder
- crass

Tried a bunch of CRISPR detection software, so far the most user friendly is Minced. 
Minced outputs a fasta file with all spacer sequences, that we can use to link MAGs to vOTUs. 
We will only be able to link MAGs that have spacers predicted. 


blast options:
and matches with ≤ 1 mismatch and an E-value ≤ 1e−5 
https://www.nature.com/articles/s41467-023-38400-0

CRISPR spacers in genome bins were BLASTn55 searched against all viral elements, with hits retained only if they had up to one mismatch, a query coverage of ≥ 90% and an E-value ≤ 10−4
https://www.nature.com/articles/s41598-023-32078-6


In [ ]:
# Do crispr regions have more snps? Use instrain
# Need to read map so make bt2 index first
srun --account=ctbrowngrp -p med2 -J bt2 -t 3:00:00 -c 24 --mem=50gb --pty bash

mamba activate bowtie2
bowtie2-build ../vOTUs/hq_virseqs.fa  -p 24 --large-index

# run instrain snake
srun --account=ctbrowngrp -p med2 -J bt2 -t 7:00:00 -c 40 --mem=80gb --pty bash

# run snakemake
mamba activate branchwater
snakemake --use-conda --resources mem_mb=80000 --rerun-triggers mtime \
-c 40 --rerun-incomplete -k -s Snakefile_instrain -n


In [ ]:
# Using Minced for CRISPR-spacers detection on the MAGs. 
# https://github.com/ctSkennerton/minced
# Total of 1482 MAGs with a crispr array
mamba activate minced

# srun
srun --account=ctbrowngrp -p med2 -J ccf -t 3:00:00 -c 24 --mem=50gb --pty bash

# run snakemake
mamba activate branchwater
snakemake --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 12 --rerunRFI_group-incomplete -k -s crispr_hostlink.smk

In [ ]:
# concatenate all spacers (n=29823)
cat *.fa > ../all_spacers.fa

# make a blastdb from all spacer file
mamba activate blast


# Make a blastdb of all vOTU sequences (no dereplication, may be interesting strain diversity))
makeblastdb -in ../hq_virseqs.fa -dbtype nucl -out hq_virseqs


# WE BLAST THE SPACER SEQUENCES TO THE VOTUS
blastn -task blastn-short \
-query all_spacers.minced.fa \
-db ../vOTUs/blastdb/hq_virseqs \
-evalue 1e-3 -perc_identity 95 -outfmt 6 \
-out spacers_vOTUs.minced

## Iphop stuff:
- Downloaded iphop db and added own new MAG sequences
- iphop needed too much time and mem, so use different approach

In case I change my mind later, here is code for adding MAGs to iphop db and running iphop

In [ ]:
# download iphop db 
# db in: /home/amhorst/databases/iphop/Aug_2023_pub_rw
mamba activate iphop_env
iphop download --db_dir ./iphop/

# run gtdbtk
# activate and run (pplacer needs mem so get scratch dir)
# Do not need classify de novo for MAGs into iphop
# can use the decorate tree command instead
mamba activate gtdbtk
gtdbtk classify_wf --cpus 36 --scratch_dir /home/amhorst/pplacer_scratch \
--genome_dir ../drep.999/dereplicated_genomes/ \
--extension fasta --skip_ani_screen --out_dir ./
# # bacteria
# gtdbtk de_novo_wf --genome_dir ../drep.999/dereplicated_genomes/ --bacteria \
# --outgroup_taxon p__Patescibacteria --out_dir pig_gut_MAGs_gtdbtk \
# --cpus 100 --force --extension fasta
# # archeae
# gtdbtk de_novo_wf --genome_dir ../drep.999/dereplicated_genomes/ --archaea \
# --outgroup_taxon p__Altiarchaeota --out_dir pig_gut_MAGs_gtdbtk \
# --cpus 100 --force --extension fasta

# decorate trees
# or can i decorate the trees instead, yes I can 
gtdbtk decorate --input_tree gtdbtk_classify/classify/gtdbtk.bac120.classify.tree.8.tree \
--output_tree gtdbtk_decorate/gtdbtk.bac120.decorated.tree.8.tree \
--gtdbtk_classification_file gtdbtk_classify/gtdbtk.bac120.summary.tsv

# made structure same as in example folder
# with an infer folder where the tree-taxonomy files are. Don't know if that works. 
# the problem is that the bacterial tree has 8 output trees. why??
# Now just using the backbone tree --> renamed and removed the word backbone. 
# with denovo MAGs, may meed to use the denovo command

In [ ]:
# add new MAGs:
# iPHoP v1.3.3
# works if you have one tree for bacteria and one tree for archeae. 
# Use the backbone tree from gtdbtk
# 
# https://bitbucket.org/srouxjgi/iphop/issues/7/no-new-spacers-in-added-custom-mags
srun --account=ctbrowngrp -p med2 -J iphop_add -t 14:00:00 -c 50 --mem=50gb --pty bash

mamba activate iphop_env

iphop add_to_db \
-f /group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/MAGs/drep.999/dereplicated_genomes/ \
-g /group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/MAGs/gtdbtk_decorate/ \
-o /home/amhorst/databases/iphop/pig_gut_MAGs_July2024/ \
-d /home/amhorst/databases/iphop/Aug_2023_pub_rw \
-t 50 

In [ ]:
# Run iphop with new MAGs added
srun --account=ctbrowngrp -p med2 -J iphop -t 100:00:00 -c 60 --mem=70gb --pty bash

mamba activate iphop_env
iphop predict -f hq_virseqs.95.cluster.fa \
-o ./iphop -d /home/amhorst/databases/iphop/pig_gut_MAGs_July2024/ -t 60